# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

import re
import numpy as np
import pandas as pd
import pickle
import seaborn as sns

from sqlalchemy import create_engine

from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# import utility to keep active in Udaicty's workspace
from workspace_utils import active_session

In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table(table_name="InsertTableName", con=engine)
X = df["message"]
y = df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):    
    # get list of all urls using regex
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # convert to lowercase
    text = text.lower()
    
    # remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # remove stop words
    tokens = [w for w in tokens if w not in stopwords.words("english")]

    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

In [5]:
# test
print(X[0])
print("    ↓    ")
print(tokenize(X[0]))

Weather update - a cold front from Cuba that could pass over Haiti
    ↓    
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        # tokenize by sentences
        sentence_list = nltk.sent_tokenize(text)
        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = nltk.pos_tag(tokenize(sentence))
            
            if len(pos_tags) > 0:
                # index pos_tags to get the first word and part of speech tag
                first_word, first_tag = pos_tags[0]
            
                # return true if the first word is an appropriate verb or RT for retweet
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return 1
                
            return 0

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)

        return pd.DataFrame(X_tagged).replace(np.nan, 0)

In [7]:
# #this pipleinie took 260s to fit. 
# pipeline = Pipeline([
#     ('features', FeatureUnion([

#         ('text_pipeline', Pipeline([
#             ('vect', CountVectorizer(tokenizer=tokenize)),
#             ('tfidf', TfidfTransformer())
#         ])),

#         ('starting_verb', StartingVerbExtractor())
#     ])),

#     ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42))) # n_estimators=10, verbose=3, n_jobs=-1
# ])

In [8]:
# # this pipleinie took 150s to fit.
# pipeline = Pipeline([
#     ('vect', CountVectorizer(tokenizer=tokenize)),
#     ('tfidf', TfidfTransformer()),
#     ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
# ])

In [9]:
# this pipleinie took 105s to fit.
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, max_features=10)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42, n_jobs=-1, verbose=3)))
])

In [10]:
# # this pipleinie took 130s to fit.
# pipeline = Pipeline([
#     ('vect', CountVectorizer(tokenizer=tokenize, max_features=100, min_df=3, max_df=0.9)),
#     ('tfidf', TfidfTransformer()),
#     ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42, n_jobs=-1)))
# ])

In [11]:
pipeline

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_a...,
            oob_score=False, random_state=42, verbose=3, warm_start=False),
           n_jobs=1))])

In [12]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
%%time
#%%prun 

# perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Fit or train the classifier
pipeline.fit(X_train, y_train)

building tree 1 of 10building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10

building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.1s remaining:    0.2s


building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10building tree 10 of 10



[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished


building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


...

[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.1s remaining:    0.2s


building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10building tree 9 of 10
building tree 10 of 10



[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.2s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.1s remaining:    0.0s


building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
CPU times: user 1min 14s, sys: 6.68 s, total: 1min 20s
Wall time: 1min 22s


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
# predict test labels
y_test_pred = pipeline.predict(X_test)

# print classification report on test data
print(classification_report(y_test.values, y_test_pred, target_names=y.columns.values))

[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:    0.0s remain

                        precision    recall  f1-score   support

               related       0.77      0.98      0.86      4938
               request       0.77      0.39      0.52      1104
                 offer       0.00      0.00      0.00        26
           aid_related       0.67      0.50      0.57      2714
          medical_help       0.25      0.01      0.03       542
      medical_products       0.48      0.04      0.07       347
     search_and_rescue       0.00      0.00      0.00       192
              security       0.00      0.00      0.00       133
              military       0.00      0.00      0.00       225
           child_alone       0.00      0.00      0.00         0
                 water       0.77      0.54      0.63       430
                  food       0.75      0.69      0.71       731
               shelter       0.33      0.03      0.06       598
              clothing       0.00      0.00      0.00       101
                 money       0.40      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [15]:
# clf_report = classification_report(y_test.values, y_test_pred, target_names=y.columns.values, output_dict=True)
# sns.heatmap(pd.DataFrame(clf_report).iloc[:-1, :].T, annot=True)

### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
%%time

# specify parameters for grid search
parameters = {
    # 'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    # 'vect__max_features': [5, 100], 
    'clf__estimator__n_estimators': [50, 100],
    # 'clf__estimator__min_samples_split': [2, 3, 4]
}

# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3, n_jobs=-1, cv=2)

# fit or train the classifier
cv.fit(X_train, y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] clf__estimator__n_estimators=50 .................................
[CV] clf__estimator__n_estimators=50 .................................
[CV] clf__estimator__n_estimators=100 ................................
[CV] clf__estimator__n_estimators=100 ................................
building tree 5 of 50
building tree 9 of 50
building tree 8 of 50
building tree 2 of 50
building tree 7 of 50
building tree 1 of 50
building tree 13 of 50
building tree 12 of 50
building tree 15 of 50
building tree 11 of 50
building tree 4 of 50
building tree 6 of 50
building tree 14 of 50
building tree 3 of 50
building tree 10 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 21 of 50
building tree 20 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 26 of 50
building tree 25 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50

[Parallel(n_jobs=-1)]: Done  36 out of  50 | elapsed:    1.5s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    1.7s finished


building tree 17 of 50
building tree 18 of 50
building tree 21 of 50
building tree 19 of 50
building tree 22 of 50
building tree 20 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 29 of 50
building tree 28 of 50
building tree 30 of 50
building tree 27 of 50
building tree 31 of 50
building tree 26 of 50
building tree 32 of 50
building tree 12 of 50
building tree 8 of 50
building tree 6 of 50
building tree 33 of 50
building tree 1 of 50
building tree 4 of 50
building tree 15 of 50
building tree 3 of 50
building tree 5 of 50
building tree 2 of 50
building tree 13 of 50
building tree 10 of 50
building tree 14 of 50
building tree 16 of 50
building tree 9 of 50
building tree 34 of 50
building tree 11 of 50
building tree 7 of 50
building tree 36 of 50
building tree 35 of 50
building tree 37 of 50
building tree 38 of 50
building tree 40 of 50
building tree 39 of 50
building tree 42 of 50
building tree 41 of 50
building tree 44 of 50
building tree 43 of 

[Parallel(n_jobs=-1)]: Done  36 out of  50 | elapsed:    1.8s remaining:    0.7s


building tree 29 of 50
building tree 28 of 50
building tree 27 of 50
building tree 31 of 50
building tree 30 of 50


...

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.7s finished


building tree 1 of 100building tree 2 of 100building tree 3 of 100building tree 4 of 100building tree 5 of 100
building tree 6 of 100
building tree 7 of 100

building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100

building tree 12 of 100

building tree 13 of 100

building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100building tree 19 of 100
building tree 20 of 100
building tree 21 of 100

building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100bu

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.1s finished


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100building tree 18 of 100

building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100building tree 23 of 100

building tree 24 of 100building tree 25 of 100building tree 26 of 100

building tree 27 of 100

building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100building tree 41 of 100
building tree 42 of 100
bu

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.4s finished


In [17]:
print("Best Parameters:", cv.best_params_)

Best Parameters: {'clf__estimator__n_estimators': 100}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
# predict test labels
y_test_pred_cv = cv.predict(X_test)

# print classification report on test data
print(classification_report(y_test.values, y_test_pred_cv, target_names=y.columns.values))

[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.1s finished
[Paral

                        precision    recall  f1-score   support

               related       0.77      0.98      0.86      4938
               request       0.76      0.40      0.52      1104
                 offer       0.00      0.00      0.00        26
           aid_related       0.67      0.50      0.57      2714
          medical_help       0.30      0.01      0.02       542
      medical_products       0.52      0.03      0.06       347
     search_and_rescue       0.00      0.00      0.00       192
              security       0.00      0.00      0.00       133
              military       0.00      0.00      0.00       225
           child_alone       0.00      0.00      0.00         0
                 water       0.79      0.52      0.62       430
                  food       0.75      0.69      0.72       731
               shelter       0.37      0.03      0.05       598
              clothing       0.00      0.00      0.00       101
                 money       0.50      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [19]:
# clf_report = classification_report(y_test.values, y_test_pred_cv, target_names=y.columns.values, output_dict=True)
# sns.heatmap(pd.DataFrame(clf_report).iloc[:-1, :].T, annot=True)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### LightGBM

In [20]:
# !pip install lightgbm==3.0.0
# from lightgbm import LGBMClassifier

### Make pipeline

In [21]:
# #pipline
# pipeline = Pipeline([
#     ('features', FeatureUnion([

#         ('text_pipeline', Pipeline([
#             ('vect', CountVectorizer(tokenizer=tokenize, max_features=10)),
#             ('tfidf', TfidfTransformer())
#         ])),

#         ('starting_verb', StartingVerbExtractor())
#     ])),

#     ('clf', MultiOutputClassifier(LGBMClassifier(random_state=42, n_jobs=-1, verbosity=2)))
# ])


In [22]:
# pipeline.get_params().keys()

### Fit

In [23]:
# %%time

# with active_session():
#     # Fit or train the classifier
#     pipeline.fit(X_train, y_train)

In [24]:
# # predict test labels
# y_test_pred = pipeline.predict(X_test)

# # print classification report on test data
# print(classification_report(y_test.values, y_test_pred, target_names=y.columns.values))

### GridSearch

In [25]:
# # specify parameters for grid search
# parameters = {
#     'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#     # 'vect__max_features': [5, 100], 
#     # 'clf__estimator__n_estimators': [50, 200],
#     # 'clf__estimator__min_samples_split': [2, 3, 4]
# }

# # create grid search object
# cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3, n_jobs=-1, cv=2)

# # fit or train the classifier
# cv.fit(X_train, y_train)

In [26]:
# # predict test labels
# y_test_pred_cv = cv.predict(X_test)

# # print classification report on test data
# print(classification_report(y_test.values, y_test_pred_cv, target_names=y.columns.values))

In [27]:
# print("Best Parameters:", cv.best_params_)

### 9. Export your model as a pickle file

In [28]:
with open("model.pkl", mode="wb") as f:
    pickle.dump(pipeline, f)

In [29]:
with open("model_cv.pkl", mode="wb") as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete ~~`train.py`~~ `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [30]:
! python ./train_classifier.py ./DisasterResponse_byScript.db ./model_byScript.pkl

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Loading data...
    DATABASE: ./DisasterResponse_byScript.db
Building model...
Training model...
Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] clf__estimator__n_estimators=50 .................................
[CV] clf__estimator__n_estimators=50 .................................
[CV] clf__estimator__n_estimators=100 ................................
[CV] clf__estimator__n_estimators=100 ................................
building tree 1 of 50
build

building tree 36 of 50
building tree 37 of 50
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 39 of 50
building tree 41 of 50
building tree 41 of 50
building tree 42 of 50
building tree 40 of 50
building tree 27 of 100
building tree 28 of 100
building tree 38 of 50
building tree 43 of 50
building tree 44 of 50
building tree 43 of 50
building tree 45 of 50
building tree 29 of 100
building tree 30 of 100
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 42 of 50
building tree 31 of 100
building tree 32 of 100
building tree 46 of 50
building tree 47 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 48 of 50
building tree 50 of 50
building tree 49 of 50
building tree 33 of 100
building tree 34 of 100
[Parallel(n_jobs=-1)]: Done  36 out of  50 | elapsed:    1.3s remaining:    0.5s
building tree 35 of 100
building tree 36 of 100
building tree 50 of 50
building

building tree 28 of 50
building tree 70 of 100
building tree 21 of 50
building tree 29 of 50
building tree 77 of 100
building tree 78 of 100
building tree 22 of 50
building tree 1 of 100
building tree 30 of 50
building tree 2 of 100
building tree 79 of 100
building tree 4 of 100
building tree 23 of 50
building tree 31 of 50
building tree 80 of 100
building tree 6 of 100
building tree 17 of 100
building tree 9 of 100
building tree 11 of 100
building tree 24 of 50
building tree 27 of 50
building tree 28 of 50
building tree 81 of 100
building tree 32 of 50
building tree 13 of 100
building tree 14 of 100
building tree 30 of 50
building tree 26 of 50
building tree 5 of 100
building tree 83 of 100
building tree 84 of 100
building tree 29 of 50
building tree 16 of 100
building tree 8 of 100
building tree 7 of 100
building tree 82 of 100
building tree 12 of 100
building tree 33 of 50
building tree 85 of 100
building tree 86 of 100
building tree 3 of 100
building tree 10 of 100
building tree 87

building tree 44 of 100
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    2.1s finished
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 1 of 50
building tree 59 of 100
building tree 2 of 50
building tree 3 of 50
building tree 6 of 50
building tree 8 of 50
building tree 10 of 50
building tree 12 of 50
building tree 60 of 100
building tree 14 of 50
building tree 61 of 100
building tree 16 of 50
building tree 62 of 100
building tree 5 of 50
building tree 9 of 50
building tree 63 of 100
building tree 13 of 50
building tree 4 of 50
building tree 11 of 50
building tree 65 of 100
building tree 64 of 100
building tree 68 of 100
building tree 66 of 100
building tree 7 of 50
building tree 70 

building tree 41 of 50
building tree 22 of 50
building tree 27 of 100
building tree 42 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 28 of 100
building tree 26 of 50
building tree 43 of 50
building tree 93 of 100
building tree 45 of 50
building tree 94 of 100
building tree 27 of 50
building tree 29 of 100
building tree 44 of 50
building tree 95 of 100
building tree 28 of 50
building tree 41 of 100
building tree 96 of 100
[Parallel(n_jobs=-1)]: Done  36 out of  50 | elapsed:    1.5s remaining:    0.6s
building tree 46 of 50
building tree 48 of 50
building tree 47 of 50
building tree 49 of 50
building tree 97 of 100
building tree 31 of 100
building tree 34 of 100
building tree 29 of 50
building tree 36 of 100
building tree 50 of 50
building tree 38 of 100
building tree 39 of 100
building tree 99 of 100
building tree 40 of 100
building tree 30 of 50
building tree 100 of 100
building tree 98 of 100
building tree 33 of 100
building tree 37 of 100
b

...

building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.8s finished
building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 6 of 100
building tree 8 of 100
building tree 10 of 100
building tree 12 of 100
building tree 14 of 100
building tree 16 of 100
building tree 4 of 100
building tree 9 of 100
building tree 13 of 100

building tree 16 of 100
building tree 11 of 100
building tree 5 of 100
building tree 14 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 33 of 100
building tree 32 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 43 of 100
building tree 42 of 100
building tree 45 of 100
building tree 44 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 

building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 56 of 100
building tree 55 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 61 of 100
building tree 60 of 100
building tree 63 of 100
building tree 64 of 100
building tree 66 of 100
building tree 65 of 100
building tree 62 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 89

building tree 83 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.3s finished
building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 5 of 100
building tree 8 of 100
building tree 10 of 100
building tree 12 of 100
building tree 14 of 100
building tree 16 of 100
building tree 6 of 100
building tree 9 of 100
building tree 13 of 100
building tree 4 of 100
building tree 11 of 100
building tree 7 of 100
building tree 15 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
b

Trained model saved!


In [31]:
with open('./model_cv.pkl', 'rb') as f:
    model_cv = pickle.load(f)

In [32]:
with open('./model_byScript.pkl', 'rb') as f:
    model_byScript = pickle.load(f)

In [33]:
model_cv

GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_a...,
            oob_score=False, random_state=42, verbose=3, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [34]:
model_byScript

GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_a...,
            oob_score=False, random_state=42, verbose=3, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [35]:
model_cv == model_byScript

False